## <font color='darkblue'>Preface</font>
([article source](https://www.attrs.org/en/stable/examples.html)) [**attrs**](https://www.attrs.org/en/stable/#) is the Python package that will bring back the joy of writing classes by relieving you from the drudgery of implementing object protocols (<font color='brown'>aka [dunder methods](https://www.attrs.org/en/latest/glossary.html#term-dunder-methods)</font>). Trusted by NASA for Mars missions since 2020!

<b>Its main goal is to help you to write concise and correct software without slowing down your code.</b>

In [145]:
import attrs
import attr
import collections
import typing
from attrs import define, field, frozen, evolve
from attrs import fields, resolve_types

<a id='sect_agenda'></a>
### <font color='darkgreen'>Agenda</font>
* <font size='3ptx'><b><a href='#sect_basics'>Basics</a></b></font>
* <font size='3ptx'><b><a href='#sect_kw_only'>Keyword-only Attributes</a></b></font>
* <font size='3ptx'><b><a href='#sect_covert_to_collections'>Converting to Collections Types</a></b></font>
* <font size='3ptx'><b><a href='#sect_defaults'>Defaults</a></b></font>
* <font size='3ptx'><b><a href='#sect_validators'>Validators</a></b></font>
* <font size='3ptx'><b><a href='#sect_conversion'>Conversions</a></b></font>
* <font size='3ptx'><b><a href='#sect_metadata'>Metadata</a></b></font>
* <font size='3ptx'><b><a href='#sect_types'>Types</a></b></font>
* <font size='3ptx'><b><a href='#sect_slots'>Slots</a></b></font>
* <font size='3ptx'><b><a href='#sect_immutability'>Immutability</a></b></font>
* <font size='3ptx'><b><a href='#sect_other_goodies'>Other Goodies</a></b></font>

<a id='sect_basics'></a>
## <font color='darkblue'>Basics</font>
The simplest possible usage is using decorator [`define`](https://www.attrs.org/en/stable/api.html#attrs.define):

In [2]:
@define
class Empty:
    pass

In [3]:
Empty()

Empty()

In [4]:
# Object equality
Empty() == Empty()

True

In [5]:
# They are two different object.
Empty() is Empty()

False

So in other words: [**attrs**](https://www.attrs.org/en/stable/#) is useful even without actual attributes! But you’ll usually want some data on your classes, so let’s add some:

In [6]:
@define
class Coordinates:
    x: int
    y: int

By default, all features are added, so you immediately have a fully functional data class with a nice repr string and comparison methods.

In [7]:
c1 = Coordinates(1, 2)
c1

Coordinates(x=1, y=2)

In [8]:
c2 = Coordinates(x=2, y=1)
c2

Coordinates(x=2, y=1)

In [9]:
c1 == c2

False

As shown, the generated `__init__` method allows for both positional and keyword arguments.

**For private attributes**, attrs will strip the leading underscores for keyword arguments:

In [10]:
@define
class C:
    _x: int

In [11]:
C(x=1)

C(_x=1)

If you want to initialize your private attributes yourself, you can do that too:

In [12]:
@define
class C:
    _x: int = field(init=False, default=42)

In [13]:
C()

C(_x=42)

In [14]:
# Below code will cause
# Traceback (most recent call last):
#   ...
# TypeError: __init__() takes exactly 1 argument (2 given)
# C(23)

An additional way of defining attributes is supported too. This is useful <b>in times when you want to enhance classes that are not yours</b> (<font color='brown'>nice `__repr__` for Django models anyone?</font>):

In [15]:
class SomethingFromSomeoneElse:
    def __init__(self, x):
        self.x = x
        
SomethingFromSomeoneElse = define(
    these={
        "x": field()
    }, init=False)(SomethingFromSomeoneElse)

In [16]:
SomethingFromSomeoneElse(1)

SomethingFromSomeoneElse(x=1)

[Subclassing is bad for you](https://www.youtube.com/watch?v=3MNVP9-hglc), but [**attrs**](https://www.attrs.org/en/stable/#) will still do what you’d hope for:
* <font color='violet'>**slots** (bool)</font> – Create a [slotted class](https://www.attrs.org/en/stable/glossary.html#term-slotted-classes) that’s more memory-efficient. Slotted classes are generally superior to the default dict classes, but have some gotchas you should know about, so we encourage you to read the [glossary entry](https://www.attrs.org/en/stable/glossary.html#term-slotted-classes).

In [17]:
@define(slots=False)
class A:
    a: int
    def get_a(self):
        return self.a
      
@define(slots=False)
class B:
    b: int
      
@define(slots=False)
class C(B, A):
    c: int

In [18]:
i = C(1, 2, 3)

In [19]:
i

C(a=1, b=2, c=3)

In [20]:
i == C(1, 2, 3)

True

In [21]:
i.get_a()

1

In [22]:
# MRO of class C
# It means we need to provide attributes in order as `a`, `b` and `c`
C.mro()

[__main__.C, __main__.B, __main__.A, object]

[**Slotted classes**](https://www.attrs.org/en/stable/glossary.html#term-slotted-classes), which are the default for the new APIs, don’t play well with multiple inheritance so we don’t use them in the example. The order of the attributes is defined by the [MRO](https://www.python.org/download/releases/2.3/mro/) (<font color='brown'>Method Resolution Order</font>).

<a id='sect_kw_only'></a>
## <font color='darkblue'>Keyword-only Attributes</font> ([back](#sect_agenda))
You can also add [keyword-only](https://docs.python.org/3/glossary.html#keyword-only-parameter) attributes:
* <font color='violet'>**kw_only** (bool)</font> – Make all attributes keyword-only (Python 3+) in the generated `__init__` (<font color='brown'>if init is False, this parameter is ignored</font>).

In [23]:
@define
class A:
    a: int = field(kw_only=True)

In [24]:
# TypeError: A.__init__() missing 1 required keyword-only argument: 'a'
# A()

In [25]:
A(a=1)

A(a=1)

`kw_only` may also be specified at via [define](https://www.attrs.org/en/stable/api.html#attrs.define), and will apply to all attributes:

In [26]:
@define(kw_only=True)
class A:
    a: int
    b: int

In [27]:
# TypeError: __init__() takes 1 positional argument but 3 were given
# A(1, 2)

In [28]:
A(a=1, b=2)

A(a=1, b=2)

If you create an attribute with <font color='blue'>init=False</font>, the <font color='violet'>kw_only</font> argument is ignored.

Keyword-only attributes allow subclasses to add attributes without default values, even if the base class defines attributes with default values:

In [29]:
@define
class A:
    a: int = 0
      
@define
class B(A):
    b: int = field(kw_only=True)

In [30]:
B(b=1)

B(a=0, b=1)

In [31]:
# B.__init__() missing 1 required keyword-only argument: 'b'
# B()

If you don’t set <font color='blue'>kw_only=True</font>, then there is no valid attribute ordering, and you’ll get an error:
```python
@define
class A:
    a: int = 0
        
@define
class B(A):
    b: int
```

Above code will incur below exception:
```python
ValueError: No mandatory attributes allowed after an attribute with a default value or factory.  Attribute in question: Attribute(name='b', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=<class 'int'>, converter=None, kw_only=False, inherited=False, on_setattr=None)
```

<a id='sect_covert_to_collections'></a>
## <font color='darkblue'>Converting to Collections Types</font> ([back](#sect_agenda))
<b>When you have a class with data, it often is very convenient to transform that class into a `dict`</b> (<font color='brown'>for example if you want to serialize it to JSON</font>). In this case **[`attrs.asdict`](https://www.attrs.org/en/stable/api.html#attrs.asdict)** may come in handy.


For example:

In [32]:
from attrs import asdict

asdict(Coordinates(x=1, y=2))

{'x': 1, 'y': 2}

Some fields cannot or should not be transformed. For that, **[`attrs.asdict`](https://www.attrs.org/en/stable/api.html#attrs.asdict)** offers a callback that decides whether an attribute should be included:

In [33]:
@define
class User:
    email: str
    password: str

@define
class UserList:
    users: list[User]

In [34]:
asdict(UserList([User("jane@doe.invalid", "s33kred"),
                 User("joe@doe.invalid", "p4ssw0rd")]),
       filter=lambda attr, value: attr.name != "password")

{'users': [{'email': 'jane@doe.invalid'}, {'email': 'joe@doe.invalid'}]}

For the common case where you want to include or exclude certain types or attributes, [**attrs**](https://www.attrs.org/en/stable/#) ships with a few helpers:

In [35]:
from attrs import asdict, filters, fields

In [36]:
@define
class User:
    login: str
    password: str
    id: int

In [37]:
asdict(
    User("jane", "s33kred", 42),
    filter=filters.exclude(fields(User).password, int))

{'login': 'jane'}

In [38]:
@define
class C:
    x: str
    y: str
    z: int

In [39]:
asdict(C("foo", "2", 3),
       filter=filters.include(int, fields(C).x))

{'x': 'foo', 'z': 3}

Other times, all you want is a tuple and [**attrs**](https://www.attrs.org/en/stable/examples.html) won’t let you down:

In [40]:
import sqlite3
from attrs import astuple

@define
class Foo:
   a: int
   b: int

In [41]:
foo = Foo(2, 3)
with sqlite3.connect(":memory:") as conn:
   c = conn.cursor()
   c.execute("CREATE TABLE foo (x INTEGER PRIMARY KEY ASC, y)") 
   c.execute("INSERT INTO foo VALUES (?, ?)", astuple(foo)) 
   foo2 = Foo(*c.execute("SELECT x, y FROM foo").fetchone())

In [42]:
foo == foo2

True

For more advanced transformations and conversions, we recommend you look at a companion library (<font color='brown'>such as [**cattrs**](https://github.com/python-attrs/cattrs)</font>).

<a id='sect_defaults'></a>
## <a href='https://www.attrs.org/en/stable/examples.html#defaults'><font color='darkblue'>Defaults</font></a> ([back](#sect_agenda))
Sometimes you want to have default values for your initializer. And sometimes you even want mutable objects as default values (<font color='brown'>ever accidentally used</font> <font color='blue'>def f(arg=[[])?</font>). [**attrs**](https://www.attrs.org/en/stable/) has you covered in both cases:
* [**class attrs.Factory**(factory, takes_self=False)](https://www.attrs.org/en/stable/api.html#attrs.Factory): Stores a factory callable.

In [48]:
@define
class Connection:
    socket: int
      
    @classmethod
    def connect(cls, db_string):
      # ... connect somehow to db_string ...
      return cls(socket=42)

@define
class ConnectionPool:
  db_string: str
  pool: collections.deque = attrs.Factory(collections.deque)
  debug: bool = False
        
  def get_connection(self):
    try:
      return self.pool.pop()
    except IndexError:
      if self.debug:
        print("New connection!")
      return Connection.connect(self.db_string)
      
  def free_connection(self, conn):
    if self.debug:
      print("Connection returned!")
      
    self.pool.appendleft(conn)

In [49]:
cp = ConnectionPool("postgres://localhost")

In [50]:
cp

ConnectionPool(db_string='postgres://localhost', pool=deque([]), debug=False)

In [51]:
conn = cp.get_connection()

In [52]:
conn

Connection(socket=42)

In [53]:
cp.free_connection(conn)

In [54]:
cp

ConnectionPool(db_string='postgres://localhost', pool=deque([Connection(socket=42)]), debug=False)

More information on why class methods for constructing objects are awesome can be found in this insightful [blog post](https://web.archive.org/web/20210130220433/http://as.ynchrono.us/2014/12/asynchronous-object-initialization.html).

Default factories can also be set using the `factory` argument to `field`, and using a decorator. The method receives the partially initialized instance which enables you to base a default value on other attributes:

In [55]:
@define
class C:
    x: int = 1
    y: int = field()
      
    @y.default
    def _any_name_except_a_name_of_an_attribute(self):
        return self.x + 1
      
    z: list = field(factory=list)

In [56]:
C()

C(x=1, y=2, z=[])

Please keep in mind that the decorator approach only works if the attribute in question has a [`field`](https://www.attrs.org/en/stable/api.html#attrs.field) assigned to it. As a result, annotating an attribute with a type is not enough if you use `@default`.

<a id='sect_validators'></a>
## <a href='https://www.attrs.org/en/stable/examples.html#validators'><font color='darkblue'>Validators</font></a> ([back](#sect_agenda))
Although your initializers should do as little as possible (<font color='brown'>ideally: just initialize your instance according to the arguments!</font>), it can come in handy to do some kind of validation on the arguments.

[**attrs**](https://www.attrs.org/en/stable/) offers two ways to define validators for each attribute and it’s up to you to choose which one suits your style and project better.

You can use a decorator:

In [58]:
@define
class C:
  x: int = field()
      
  @x.validator
  def check(self, attribute, value):
    if value > 42:
      raise ValueError("x must be smaller or equal to 42")

In [59]:
C(42)

C(x=42)

In [61]:
# ValueError: x must be smaller or equal to 42
# C(43)

…or a callable…

In [62]:
from attrs import validators

def x_smaller_than_y(instance, attribute, value):
  if value >= instance.y:
    raise ValueError("'x' has to be smaller than 'y'!")
        
@define
class C:
  x: int = field(validator=[validators.instance_of(int),
                            x_smaller_than_y])
  y: int

In [63]:
C(x=3, y=4)

C(x=3, y=4)

In [65]:
# ValueError: 'x' has to be smaller than 'y'!
# C(x=4, y=3)

…or both at once:

In [66]:
@define
class C:
  x: int = field(validator=validators.instance_of(int))

  @x.validator
  def fits_byte(self, attribute, value):
    if not 0 <= value < 256:
      raise ValueError("value out of bounds")

In [67]:
C(128)

C(x=128)

In [69]:
# TypeError: ("'x' must be <class 'int'> (got '128' that is a <class 'str'>)."
# C("128")

In [71]:
# ValueError: value out of bounds
# C(256)

Please note that the decorator approach only works if – and only if! – the attribute in question has a [`field`](https://www.attrs.org/en/stable/api.html#attrs.field) assigned. Therefore if you use `@validator`, it is not enough to annotate said attribute with a type.

[**attrs**](https://www.attrs.org/en/stable/) ships with a bunch of [validators](https://www.attrs.org/en/stable/init.html#validators), make sure to check them out before writing your own:

In [72]:
@define
class C:
  x: int = field(validator=validators.instance_of(int))

In [74]:
# TypeError: ("'x' must be <class 'int'> (got '42' that is a <class 'str'>)."
# C("42")

Please note that if you use [`attr.s`](https://www.attrs.org/en/stable/api.html#attr.s) (<font color='brown'>and not [`attrs.define`](https://www.attrs.org/en/stable/api.html#attrs.define)</font>) to define your class, validators only run on initialization by default. This behavior can be changed using the <font color='violet'>on_setattr</font> argument.

Check out [**Validators**](https://www.attrs.org/en/stable/init.html#validators) for more details.

<a id='sect_conversion'></a>
## <a href='https://www.attrs.org/en/stable/examples.html#conversion'><font color='darkblue'>Conversion</font></a> ([back](#sect_agenda))
Attributes can have a `converter` function specified, which will be called with the attribute’s passed-in value to get a new value to use. This can be useful for doing type-conversions on values that you don’t want to force your callers to do.

In [75]:
@define
class C:
  x: int = field(converter=int)

In [77]:
o = C("1")
print(f'o.x = {o.x} ({o.x.__class__})')

o.x = 1 (<class 'int'>)


Please note that <b>converters only run on initialization</b>. Check out [**Converters**](https://www.attrs.org/en/stable/init.html#converters) for more details.

<a id='sect_metadata'></a>
## <a href='https://www.attrs.org/en/stable/examples.html#metadata'><font color='darkblue'>Metadata</font></a> ([back](#sect_agenda))
All [**attrs**](https://www.attrs.org/en/stable/) attributes may include arbitrary metadata in the form of a read-only dictionary.

In [78]:
from attrs import fields

@define
class C:
  x = field(metadata={'my_metadata': 1})

In [79]:
fields(C).x.metadata

mappingproxy({'my_metadata': 1})

In [80]:
fields(C).x.metadata['my_metadata']

1

Metadata is not used by [**attrs**](https://www.attrs.org/en/stable/), and is meant to enable rich functionality in third-party libraries. The metadata dictionary follows the normal dictionary rules: keys need to be hashable, and both keys and values are recommended to be immutable.

If you’re the author of a third-party library with [**attrs**](https://www.attrs.org/en/stable/) integration, please see [Extending Metadata](https://www.attrs.org/en/stable/extending.html#extending-metadata).

<a id='sect_types'></a>
## <a href='https://www.attrs.org/en/stable/examples.html#types'><font color='darkblue'>Types</font></a> ([back](#sect_agenda))
[**attrs**](https://www.attrs.org/en/stable/) also allows you to associate a type with an attribute using either the `type` argument to [attr.ib](https://www.attrs.org/en/stable/api.html#attr.ib) or – as of Python 3.6 – using [**PEP 526-annotations**](https://peps.python.org/pep-0526/):

In [82]:
@define
class C:
  x: int

In [83]:
fields(C).x.type

int

In [86]:
@attr.s
class C:
  x = attr.ib(type=int)

In [87]:
fields(C).x.type

int

If you don’t mind annotating all attributes, you can even drop the [attrs.field](https://www.attrs.org/en/stable/api.html#attrs.field) and assign default values instead:

In [92]:
@define
class AutoC:
  cls_var: typing.ClassVar[int] = 5  # this one is ignored
  l: list[int] = attrs.Factory(list)
  x: int = 1
  foo: str = "every attrib needs a type if auto_attribs=True"
  bar: typing.Any = None

In [93]:
fields(AutoC).l.type

list[int]

In [94]:
fields(AutoC).x.type

int

In [95]:
fields(AutoC).foo.type

str

In [96]:
fields(AutoC).bar.type

typing.Any

In [97]:
AutoC()

AutoC(l=[], x=1, foo='every attrib needs a type if auto_attribs=True', bar=None)

In [98]:
AutoC.cls_var

5

The generated `__init__` method will have an attribute called `__annotations__` that contains this type information.

If your annotations contain strings (<font color='brown'>e.g. forward references</font>), you can resolve these after all references have been defined by using [attrs.resolve_types()](https://www.attrs.org/en/stable/api.html#attrs.resolve_types). This will replace the type attribute in the respective fields.

In [100]:
@define
class A:
    a: 'list[A]'
    b: 'B'

@define
class B:
    a: A

In [101]:
fields(A).a.type

'list[A]'

In [102]:
fields(A).b.type

'B'

In [103]:
resolve_types(A, globals(), locals())

__main__.A

In [104]:
fields(A).a.type

list[__main__.A]

In [105]:
fields(A).b.type

__main__.B

<a id='sect_slots'></a>
## <a href='https://www.attrs.org/en/stable/examples.html#slots'><font color='darkblue'>Slots</font></a> ([back](#sect_agenda))
[**Slotted classes**](https://www.attrs.org/en/stable/glossary.html#term-slotted-classes) have several advantages on CPython. Defining `__slots__` by hand is tedious, in [**attrs**](https://www.attrs.org/en/stable/) it’s just a matter of using [attrs.define](https://www.attrs.org/en/stable/api.html#attrs.define) or passing <font color='blue'>slots=True</font> to [attr.s](https://www.attrs.org/en/stable/api.html#attr.s):

In [130]:
@attr.s(slots=True, auto_attribs=True)
class Person:
  name: str
  age: int

In [131]:
p = Person(name='John', age=42)

In [138]:
# slots = True doesn't allow you to create new attribute
# 'Person' object has no attribute 'height'
# p.height=10

In [135]:
@attr.s(auto_attribs=True)
class Person:
  name: str
  age: int

In [136]:
p = Person(name='John', age=42)

In [137]:
p.height=10

<a id='sect_immutability'></a>
## <a href='https://www.attrs.org/en/stable/examples.html#immutability'><font color='darkblue'>Immutability</font></a> ([back](#sect_agenda))
Sometimes you have instances that shouldn’t be changed after instantiation. Immutability is especially popular in functional programming and is generally a very good thing. If you’d like to enforce it, [**attrs**](https://www.attrs.org/en/stable/) will try to help:

In [141]:
@frozen
class C:
  x: int

In [142]:
i = C(1)

In [144]:
# FrozenInstanceError: 
# i.x = 2

Please note that true immutability is impossible in Python but it will [get you 99% there](https://www.attrs.org/en/stable/how-does-it-work.html#how-frozen). By themselves, immutable classes are useful for long-lived objects that should never change; like configurations for example.

In order to use them in regular program flow, you’ll need a way to easily create new instances with changed attributes. In Clojure that function is called [assoc](https://clojuredocs.org/clojure.core/assoc) and [**attrs**](https://www.attrs.org/en/stable/) shamelessly imitates it: [attr.evolve](https://www.attrs.org/en/stable/api.html#attr.attr.evolve):

In [147]:
@frozen
class C:
  x: int
  y: int

In [148]:
i1 = C(1, 2)

In [149]:
i1

C(x=1, y=2)

In [150]:
i2 = evolve(i1, y=3)

In [151]:
i2

C(x=1, y=3)

In [152]:
i1 == i2

False

<a id='sect_other_goodies'></a>
## <a href='https://www.attrs.org/en/stable/examples.html#other-goodies'><font color='darkblue'>Other Goodies</font></a> ([back](#sect_agenda))
Sometimes you may want to create a class programmatically. [**attrs**](https://www.attrs.org/en/stable/) won’t let you down and gives you [attrs.make_class](https://www.attrs.org/en/stable/api.html#attrs.make_class) :
> A quick way to create a new class called name with attrs.

In [153]:
from attrs import fields, make_class

@define
class C1:
  x = field()
  y = field()

In [154]:
C2 = make_class("C2", ["x", "y"])

In [155]:
fields(C1) == fields(C2)

True

You can still have power over the attributes if you pass a dictionary of name: `field` mappings and can pass arguments to <font color='orange'><b>@attr.s</b></font>:

In [157]:
C = make_class("C", {"x": field(default=42),
                     "y": field(default=attrs.Factory(list))},
               repr=False)

In [158]:
i = C()

In [159]:
i  # no repr added!

In [160]:
i.x

42

In [161]:
i.y

[]

If you need to dynamically make a class with [attrs.make_class](https://www.attrs.org/en/stable/api.html#attrs.make_class) and it needs to be a subclass of something else than `object`, use the <font color='violet'>bases</font> argument:

In [162]:
class D:
  def __eq__(self, other):
    return True  # arbitrary example

In [164]:
# Setting cmp is equivalent to setting eq and order to the same value. Must not be mixed with eq or order.
C = make_class("C", {}, bases=(D,), cmp=False)

In [165]:
isinstance(C(), D)

True

Sometimes, you want to have your class’s `__init__` method do more than just the initialization, validation, etc. that gets done for you automatically when using `@define`. To do this, just define a `__attrs_post_init__` method in your class. It will get called at the end of the generated `__init__` method.

In [166]:
@define
class C:
  x: int
  y: int
  z: int = field(init=False)

  def __attrs_post_init__(self):
    self.z = self.x + self.y

In [167]:
obj = C(x=1, y=2)

In [168]:
obj

C(x=1, y=2, z=3)

You can exclude single attributes from certain methods:
* `repr (bool) – Create a __repr__ method with a human readable representation of attrs attributes.`

In [169]:
@define
class C:
  user: str
  password: str = field(repr=False)

In [170]:
C("me", "s3kr3t")

C(user='me')

Alternatively, to influence how the generated `__repr__()` method formats a specific attribute, specify a custom callable to be used instead of the [repr()](https://docs.python.org/3/library/functions.html#repr) built-in function:

In [171]:
@define
class C:
  user: str
  password: str = field(repr=lambda value: '***')

In [172]:
C("me", "s3kr3t")

C(user='me', password=***)

## <font color='darkblue'>Supplement</font>
* [Why not - dataclasses?](https://www.attrs.org/en/stable/why.html#data-classes)
* [Python- `__slots__` 變量介紹](https://medium.com/@yaia52009/%E9%BC%A0%E5%B9%B4%E5%85%A8%E9%A6%AC%E9%90%B5%E4%BA%BA%E6%8C%91%E6%88%B0week9-python-slots-%E8%AE%8A%E9%87%8F-9248d08c12a6)
> Python是一種動態語言, 通常動態語言允許我們在程式運行時給物件綁定心的屬性或方法, 當然也可以對已經綁定屬性的方法解除綁定, 但是如果我們需要限定自定義類的物件只能綁定某些屬性, 可以通過在類中定義 `__slots__` 變量來進行限定, 需要注意的是 `__slots__` 的限定只對當前類的物件生效, 對子類不起任何作用...